### Breach Data Preparation as per the Business Logic

In [17]:
import warnings
from IPython.display import display, HTML
import sys,os
import pandas as pd
import copy
sys.path.insert(0, '../scripts/')
from nlp_preprocess import * 

warnings.filterwarnings("ignore")
%store -r complaints_df

In [19]:
#!pip install --user imblearn
breach_df = complaints_df[['ConcernID','IssueID','Breach','IssueDescription','ConcernDescription','ResolutionDescription','Clause']]
breach_df.Breach= breach_df.Breach.str.replace(' ', '')
breach_df.Breach= breach_df.Breach.str.lower()

In [20]:
breach_df.Breach.value_counts()

no          31301
possible     3346
yes            74
Name: Breach, dtype: int64

In [21]:
breach_df.loc[breach_df['Breach'] == 'yes', 'Breach'] = 'possible'
breach_df.Breach.value_counts()

no          31301
possible     3420
Name: Breach, dtype: int64

In [22]:
breach_df['Breach'] = breach_df['Breach'].map({'possible': 1, 'no': 0})

In [23]:
result = breach_df.groupby(['Clause','Breach']).size()
df = pd.DataFrame(result)

In [24]:
# df.reset_index()
breach_df_copy = breach_df.copy()
breach_df_copy = pd.get_dummies(breach_df_copy, columns=['Breach'], prefix = ['Breach'])

In [25]:
#breach_df_copy.set_index('Clause',inplace=True)
breach_df_copy.drop(breach_df_copy.index[34721:],inplace=True)

In [26]:
df = pd.DataFrame(breach_df_copy.groupby(['Clause']).sum())
df['Total'] = df['Breach_0.0']+ df['Breach_1.0']
df['Breach_Prop'] = df['Breach_1.0']/ df['Total']
temp_df = df.sort_values('Breach_Prop',ascending=False)
sort_df = df.sort_values('Total',ascending=False)

In [27]:
multi_table([sort_df.head(),temp_df.head()],HTML=HTML)

,Breach_0.0,Breach_1.0,Total,Breach_Prop
Clause,,,,
14a,8695.0,984.0,9679.0,0.101663
33,4401.0,1056.0,5457.0,0.193513
15,2839.0,314.0,3153.0,0.099588
18.1,2033.0,5.0,2038.0,0.002453
22,1855.0,30.0,1885.0,0.015915
,Breach_0.0,Breach_1.0,Total,Breach_Prop
Clause,,,,
4,0.0,6.0,6.0,1.000000
24,151.0,116.0,267.0,0.434457


In [28]:
final_df = df.sort_values('Breach_1.0',ascending=False).head(4)
final_df

,Breach_0.0,Breach_1.0,Total,Breach_Prop
Clause,,,,
33,4401.0,1056.0,5457.0,0.193513
14a,8695.0,984.0,9679.0,0.101663
15,2839.0,314.0,3153.0,0.099588
18.2,622.0,171.0,793.0,0.215637


In [29]:
final_df.sum()

Breach_0.0     16557.000000
Breach_1.0      2525.000000
Total          19082.000000
Breach_Prop        0.610401
dtype: float64

In [30]:
final_df.sum()[0]/final_df.sum()[2]

0.8676763441987213

### Prepare dataset for the 4 dominant classes for breach modelling

In [31]:
final_clauses = list(final_df.index)
final_clauses

['33', '14a', '15', '18.2']

In [32]:
reduced_breach_df = breach_df.loc[breach_df['Clause'].isin (final_clauses)]

In [33]:
reduced_breach_df.head()

,ConcernID,IssueID,Breach,IssueDescription,ConcernDescription,ResolutionDescription,Clause
15,969994,1020622,1.0,lack of information,Customer has an opened an ANZ Offset account o...,I reconfirmed his concerns. I apologised to cu...,15
34,970497,1021093,1.0,Insufficient information provided.,Customer opened an ANZ Progress Saver account ...,Customer Advocate: offered to reimburse the fi...,15
41,970854,1021667,0.0,incorrect information provided,Your ANZ Online Saver account was opened at AN...,Thank you for the opportunity to investigate t...,15
49,971112,1020452,0.0,account opened in different name than business...,you visited ANZ Bourke Street Mall branch on 1...,Thank you for the opportunity to investigate t...,15
63,971462,1022374,0.0,cheque alteration,Customer's ANZ V2Plus account has been closed ...,We are unable to alter the cheque in your pers...,33


In [34]:
reduced_breach_df['keyword_corpus'] = reduced_breach_df['ConcernDescription']+"."+reduced_breach_df['IssueDescription']+"."+reduced_breach_df['ResolutionDescription']
reduced_breach_df['keyword_corpus'] = reduced_breach_df[['ConcernDescription','IssueDescription','ResolutionDescription','Clause']].fillna('').sum(axis=1)
reduced_breach_df.head()


,ConcernID,IssueID,Breach,IssueDescription,ConcernDescription,ResolutionDescription,Clause,keyword_corpus
15,969994,1020622,1.0,lack of information,Customer has an opened an ANZ Offset account o...,I reconfirmed his concerns. I apologised to cu...,15,Customer has an opened an ANZ Offset account o...
34,970497,1021093,1.0,Insufficient information provided.,Customer opened an ANZ Progress Saver account ...,Customer Advocate: offered to reimburse the fi...,15,Customer opened an ANZ Progress Saver account ...
41,970854,1021667,0.0,incorrect information provided,Your ANZ Online Saver account was opened at AN...,Thank you for the opportunity to investigate t...,15,Your ANZ Online Saver account was opened at AN...
49,971112,1020452,0.0,account opened in different name than business...,you visited ANZ Bourke Street Mall branch on 1...,Thank you for the opportunity to investigate t...,15,you visited ANZ Bourke Street Mall branch on 1...
63,971462,1022374,0.0,cheque alteration,Customer's ANZ V2Plus account has been closed ...,We are unable to alter the cheque in your pers...,33,Customer's ANZ V2Plus account has been closed ...


In [ ]:
breach_final_df = reduced_breach_df.copy()
%store breach_final_df

In [ ]:
# def get_train_texts():
#     train_texts = []
#     try:
#         for index,line in enumerate(reduced_breach_df['keyword_corpus']):
#             tokens = tokenizer(line)
#             tokenz = [val for i, val in enumerate(tokens) if val not in custom_stopwords]
#             #tokens = remove_custom_stopwords(line,custom_stopwords)
#             train_texts.append(tokenz)
#         return train_texts

#     except Exception as e:
#         print(index,e)
    
    

In [ ]:
# import time
# start = time.time()
# train_texts = get_train_texts()
# phrased = preprocess_phrase(train_texts)
# print((time.time()-start))

In [ ]:
# phrased_final =[]
# from gensim.parsing.preprocessing import strip_multiple_whitespaces,strip_short
# for line in phrased:
#     tmp =[]
#     for each in line:
#         if each not in my_stop_words:
#             tmp.append(strip_short(each))
#     phrased_final.append(tmp)
    

In [ ]:
# reduced_breach_df['final_tokens'] = phrased_final

In [ ]:
# reduced_breach_df.head()

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer
# from sklearn.metrics import confusion_matrix, classification_report, accuracy_score,recall_score
# from sklearn.model_selection import train_test_split
# from sklearn.linear_model import LogisticRegression
# from sklearn.model_selection import GridSearchCV
# from sklearn.ensemble import GradientBoostingClassifier , RandomForestClassifier
# from imblearn.over_sampling import SMOTE
# #from sklearn.pipeline import Pipeline
# from imblearn.pipeline import Pipeline

In [ ]:
# tfidf = TfidfVectorizer(tokenizer=lambda x: x, lowercase=False)
# Y = reduced_breach_df['Breach']
# X_train, X_test, Y_train, Y_test = train_test_split(phrased_final, Y, test_size=0.20, random_state=42)

In [ ]:
# Y_train.value_counts()

In [ ]:
# X_train = tfidf.fit_transform(X_train)

# X_test = tfidf.transform(X_test)

In [ ]:
# Y_test.value_counts()

In [ ]:
# smt = SMOTE(random_state=42,ratio=0.95)
# x_train, y_train = smt.fit_sample(X_train, Y_train)

In [ ]:
# import numpy as np
# print(X_train.shape,Y_train.shape)
# print(x_train.shape,y_train.shape)

In [ ]:
# clf = LogisticRegression()

In [ ]:
# clf.fit(x_train, y_train)


In [ ]:
#x_test, y_test = smt.fit_sample(X_test, Y_test)
# pred = clf.predict(X_test)

In [ ]:
# print("Recall {}".format(recall_score(Y_test,pred)))
# print("Accuracy {}".format(accuracy_score(pred,Y_test)))